In [1]:
import numpy as np
import random
import itertools
import collections

def to_tuple_if_not(pos):
    return pos if isinstance(pos, tuple) else tuple(pos)
    
def vadd_tuple(t1, t2):
    return tuple(x1 + x2 for x1, x2 in zip(t1, t2))
    
class LifeGame:
    def __init__(self, shape=(24, 40), *, periodic=True, rule=None):
        self.periodic = periodic
        self.rule = rule if rule else {True: (2, 3), False: (3,)} # 23/3
        self.generation = 0
        self.shape = to_tuple_if_not(shape)
        self.dimension = len(self.shape)
        self.board = np.zeros(self.shape, dtype=bool)
    
    def __getitem__(self, key):
        key = to_tuple_if_not(key)
        if self.periodic:
            return self.board[tuple(x % n for (x, n) in zip(key, self.shape))]
        elif all(0 <= x < n for (x, n) in zip(key, self.shape)):
            return self.board[key]
        else:
            return False
            
    def __setitem__(self, key, value):
        key = to_tuple_if_not(key)
        if self.periodic:
            self.board[tuple(x % n for (x, n) in zip(key, self.shape))] = value
        elif all(0 <= x < n for (x, n) in zip(key, self.shape)):
            self.board[key] = value
        else:
            pass
    
    def keys(self):
        nranges = (range(n) for n in self.shape)
        return itertools.product(*nranges)
    
    def count_neighbor(self, key):
        dranges = itertools.repeat(range(-1, 2), self.dimension)
        dkeys = itertools.product(*dranges)
        dkeys = filter(any, dkeys) # remove (0, 0, ...)
        neighbors = (vadd_tuple(key, dkey) for dkey in dkeys)
        return sum(self[neighbor] for neighbor in neighbors)
        
    def next_cell_state(self, key):
        ns = self.rule[self[key]]
        return self.count_neighbor(key) in ns
        
    def randomize(self, randobj=None):
        randobj = randobj if randobj else random
        for key in self.keys():
            self[key] = bool(randobj.randint(0, 1))            
    
    def __str__(self):
        queue = collections.deque()
        queue.append(self.board)
        s = ''
        while queue:
            item = queue.popleft()
            if isinstance(item, str):
                s += item
            elif len(item.shape) == 1:
                s += ''.join(('██' if x else '  ') for x in item)
            else:
                for i, x in enumerate(item):
                    if i != 0:
                        queue.append('\n')
                    queue.append(x)
        return s
    
    def __next__(self):
        next_board = np.zeros(self.shape, dtype=bool)
        for key in self.keys():
            next_board[key] = self.next_cell_state(key)
        self.board = next_board
        return self

    def __iter__(self):
        return self

In [2]:
lg = LifeGame((10,10), periodic=True)
lg.randomize()

In [3]:
print(next(lg))

██  ██  ██████  ██  
██                  
██        ██      ██
          ██        
        ██      ██  
    ██    ██    ██  
        ████        
    ████          ██
      ██  ██    ██  
  ████          ████
